In [1]:
## General librairies
import pandas as pd
import numpy as np
import _pickle as pickle
import matplotlib.pyplot as plt
import random
import sys
import time
from copy import deepcopy
import os

from sklearn.metrics import cohen_kappa_score

from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import json

import dateutil.parser

Using TensorFlow backend.


In [2]:
specs = 'specs.csv'
test = 'test.csv'
train = 'train.csv'
train_labels = 'train_labels.csv'
subs = 'sample_submission_exemple.csv'

In [3]:
df = pd.read_csv(train)

In [4]:
def save(file,name, folder = ""):
    if folder != "":
        outfile = open('./'+folder+'/'+name+'.pickle', 'wb')
    else:
        outfile = open(name+'.pickle', 'wb')
    pickle.dump(file, outfile)
    outfile.close
    
def load(name, folder = ""):
    if folder != "":
        outfile = open('./'+folder+'/'+name+'.pickle', 'rb')
    else:
        outfile = open(name+'.pickle', 'rb')
    file = pickle.load(outfile)
    outfile.close
    return file

def relative_time(x):
    x1 = []
    for elt in x:
        x1.append((elt-x[0]).item()/1000000000)
    return x1

def categorise(x):
    dico = {}
    count = 0
    for elt in x:
        if not(elt in dico):
            dico[elt] = count
            count += 1
    return dico

def padding( dataset, n):
    d = list(np.zeros(len(dataset)))
    c = 0
    count = 0
    for elt in dataset:
        elt = elt.values.astype('float32')
        if count % 100 == 0:
            print(count)
        u = elt.shape[0]
        
        if u<200:
            c += 1
        
        if u > n:
            d[count] = elt[-n:]
        else:
            a = np.zeros(((n-u), elt.shape[1])) -1
            elt = np.concatenate([a, elt], axis = 0)
            d[count] = elt
        count += 1
    return d

def build_line(df, installation_id, game_session):
    
    df1 = df[(df['installation_id']==installation_id)&(df['date']<=df[(df['installation_id']==installation_id)&(df['game_session']==game_session)]['date'].iloc[0])]
    dico = categorise(df1['game_session'].unique())
    df1 = df1.replace({'game_session' : dico})
    x = relative_time(df1['date'].values)
    
    df1['time_delta'] = x
    
#     df1 = df1.drop_duplicates(subset = ['event_id'])
    
    cats_to_keep = ['game_session','time_delta','title','type', 'world',  'event_count', 'game_time', 'event_code']
    df1 = df1[cats_to_keep]
    return df1

def format_input(X):
    return [X[:,:,4:24], X[:,:,0].reshape((X.shape[0], X.shape[1],1)), 
            X[:,:,1].reshape((X.shape[0], X.shape[1],1)), 
            X[:,:,2].reshape((X.shape[0], X.shape[1],1)),
            X[:,:,3].reshape((X.shape[0], X.shape[1],1)),
            X[:,:,24].reshape((X.shape[0], X.shape[1],1))]

def build_feature(i,session):
    vect = []
    
    title = session.iloc[0]['title']
    typ = session.iloc[0]['type']
    
    vect.append(session.iloc[0]['game_session'])    ##game_session
    vect.append(session.iloc[0]['installation_id'])   ##installation_id
    vect.append(session.iloc[0]['title'])  ##title
    vect.append(session.iloc[0]['type'])  ##type
    vect.append(session.iloc[0]['world'])  ##world
    vect.append(session.iloc[0]['timestamp'])  ##timestamp
    
    
    ## adding validation data if game or assessment
    if typ == 'Game' or typ == 'Assessment':
        pos = 0
        neg = 0
        for elt in session[session['event_code']==4100]['event_data']:
            a = json.loads(elt)['correct']
#             print(a)
            if a:
                pos+=1
            else:
                neg+=1
            
        if pos+neg ==0:
            acc = -1
        else:
            acc = pos/(pos+neg)
        
#         print(acc)
        if acc == 1:
            acc_class = 3
        elif acc == 0.5:
            acc_class = 2
        elif acc <=0:
            acc_class = 0
        else:
            acc_class = 1
#         print(acc_class)
        vect.append(acc_class)   ## accuracy_group
        vect.append(acc)       ## accuracy
        vect.append(pos)       ## n_positive
        vect.append(neg)       ## n_negative
            
            
    else: 
        vect.append(-1)  ##accuracy_group
        vect.append(-1) ## accuracy_group
        vect.append(-1)  ## n_positive
        vect.append(-1)  ## n_negative
    
    ## Adding features relative to Game, Assessment and activity
    if typ != 'Clip':
        ## Avg time per instruction
        
        st = session['game_time'][session['event_code'] == 3010]
        en = session['game_time'][session['event_code'] == 3110]
        
        s = min(st.shape[0], en.shape[0])
        if s!=0:
            vect.append((en[:s].sum()-st[:s].sum())/s)  ## time per instruction
        else:
            vect.append(-1)  ##time per instruction
        
        ##total_time
        total_time = session['game_time'].max()/1000
        vect.append(total_time)
        
        ##number of actions
        n_action = session[(session['event_code']>=4020)&(session['event_code']<=4080)].shape[0]
        vect.append(n_action)
        
        ## rounds
        n_rounds_st = session[session['event_code']==2020].shape[0]
        n_rounds_en = session[session['event_code']==2030].shape[0]
        
        vect.append(n_rounds_st)
        vect.append(n_rounds_st)
        
        if n_rounds_st != 0:
            vect.append(n_action/n_rounds_st)
            vect.append(total_time/n_rounds_st)
        else:
            vect.append(-1)
            vect.append(-1)
        ## hints
        vect.append(session[session['event_code']==4090].shape[0])
        
    else:
        vect.append(-1)  ## time per instruction
        vect.append(-1)  ##  total time
        vect.append(-1)  ## n_actions
        vect.append(-1)  ## n_rounds_start
        vect.append(-1)  ## n_round end
        vect.append(-1)  ## action per round
        vect.append(-1)  ## time per round
        vect.append(-1)  ## N_hints
    
    if typ == 'Game' or typ == 'Assessment':
        ## feed backs
        
        n_correct_f = session[session['event_id'] == 3121].shape[0]
        n_incorrect_f = session[session['event_id'] == 3120].shape[0]
        
        vect.append(n_correct_f)
        vect.append(n_incorrect_f)
        
        if n_incorrect_f + n_correct_f != 0:
            vect.append(n_correct_f/(n_correct_f+n_incorrect_f))
        else:
            vect.append(0)
        
        if n_correct_f !=0:
            vect.append((session['game_time'][session['event_code'] == 3121].sum()-session['game_time'][session['event_code'] == 3021].sum())/n_correct_f)
        else:
            vect.append(-1)
        
        if n_incorrect_f !=0:
            vect.append((session['game_time'][session['event_code'] == 3120].sum()-session['game_time'][session['event_code'] == 3020].sum())/n_correct_f)
        else:
            vect.append(-1)
    else:
        vect.append(-1)  ## correct_feed
        vect.append(-1)  ## incorrect_feed
        vect.append(-1)  ## acc_feed
        vect.append(-1)  ## time correct feedback
        vect.append(-1)  ## time incorrect feedback
    
    if typ == 'Game':
        vect.append(session[session['event_code'] == 2080].shape[0])
        vect.append(session[session['event_code'] == 2081].shape[0])
        vect.append(session[session['event_code'] == 2060].shape[0])
        vect.append(session[session['event_code'] == 2075].shape[0])
        
        
    else:
        vect.append(-1)  ## n movie
        vect.append(-1)  ## n_skipp movie
        vect.append(-1)  ## tutorial
        vect.append(-1)  ## skipped tutorial
        
    
    
    return vect

titles = df['title'].unique()
def build_set(df, installation_id, game_session):
    df1 = df[df['installation_id']==installation_id]
    
    df1 = df1.sort_values(by = ['date'], ascending = True)
    date = df1[df1['game_session']==game_session].iloc[0]['date']
    pred_title = df1[df1['game_session']==game_session].iloc[0]['title']
    
    df1['pred_title'] = pred_title
    df1 = df1[df1['date'] < date]
    
    
    vect = []
    
    vect.append(pred_title)
    
    ## Clip
    df2 = df1[df1['type']=='Clip']
    vect.append(df2.shape[0])
    
    ## Activity
    df2 = df1[df1['type']=='Activity']
    
    #n_activity
    vect.append(df2.shape[0])
    # instruction
    vect.append(df2[df2['time_instruction']!=-1]['time_instruction'].values.astype('float32').mean())
    vect.append(df2[df2['total_time']!=-1]['total_time'].values.astype('float32').mean())
    vect.append(df2[df2['actions']!=-1]['actions'].values.astype('float32').mean())
    vect.append(df2[df2['start_rounds']!=-1]['start_rounds'].values.astype('float32').mean())
    vect.append(df2[df2['end_rounds']!=-1]['end_rounds'].values.astype('float32').mean())
    vect.append(df2[df2['action_rounds']!=-1]['action_rounds'].values.astype('float32').mean())
    vect.append(df2[df2['time_rounds']!=-1]['time_rounds'].values.astype('float32').mean())
    vect.append(df2[df2['hints']!=-1]['hints'].values.astype('float32').mean())
    
    ## Assessment
    df2 = df1[df1['type']=='Assessment']
    if pred_title in df2['title'].values:
        vect.append(1)
    else:
        vect.append(0)
    vect.append(df2.shape[0])
    vect.append(df2[df2['accuracy_group']!=-1]['accuracy_group'].values.astype('float32').mean())
    vect.append(df2[df2['time_instruction']!=-1]['time_instruction'].values.astype('float32').mean())
    vect.append(df2[df2['total_time']!=-1]['total_time'].values.astype('float32').mean())
    vect.append(df2[df2['actions']!=-1]['actions'].values.astype('float32').mean())
    vect.append(df2[df2['start_rounds']!=-1]['start_rounds'].values.astype('float32').mean())
    vect.append(df2[df2['end_rounds']!=-1]['end_rounds'].values.astype('float32').mean())
    vect.append(df2[df2['action_rounds']!=-1]['action_rounds'].values.astype('float32').mean())
    vect.append(df2[df2['time_rounds']!=-1]['time_rounds'].values.astype('float32').mean())
    vect.append(df2[df2['hints']!=-1]['hints'].values.astype('float32').mean())
    vect.append(df2[df2['correct_feed']!=-1]['correct_feed'].values.astype('float32').mean())
    vect.append(df2[df2['incorrect_feed']!=-1]['incorrect_feed'].values.astype('float32').mean())
    vect.append(df2[df2['time_corr_feed']!=-1]['time_corr_feed'].values.astype('float32').mean())
    vect.append(df2[df2['time_inc_feed']!=-1]['time_inc_feed'].values.astype('float32').mean())
    vect.append(df2[df2['acc_feed']!=-1]['acc_feed'].values.astype('float32').mean())
    vect.append(df2[df2['accuracy']!=-1]['accuracy'].values.astype('float32').mean())
    vect.append(df2[df2['n_positive']!=-1]['n_positive'].values.astype('float32').mean())
    vect.append(df2[df2['n_negative']!=-1]['n_negative'].values.astype('float32').mean())
    
    ## Games
    df2 = df1[df1['type']=='Game']
    vect.append(df2.shape[0])
    vect.append(df2[df2['accuracy_group']!=-1]['accuracy_group'].values.astype('float32').mean())
    vect.append(df2[df2['time_instruction']!=-1]['time_instruction'].values.astype('float32').mean())
    vect.append(df2[df2['total_time']!=-1]['total_time'].values.astype('float32').mean())
    vect.append(df2[df2['actions']!=-1]['actions'].values.astype('float32').mean())
    vect.append(df2[df2['start_rounds']!=-1]['start_rounds'].values.astype('float32').mean())
    vect.append(df2[df2['end_rounds']!=-1]['end_rounds'].values.astype('float32').mean())
    vect.append(df2[df2['action_rounds']!=-1]['action_rounds'].values.astype('float32').mean())
    vect.append(df2[df2['time_rounds']!=-1]['time_rounds'].values.astype('float32').mean())
    vect.append(df2[df2['hints']!=-1]['hints'].values.astype('float32').mean())
    vect.append(df2[df2['correct_feed']!=-1]['correct_feed'].values.astype('float32').mean())
    vect.append(df2[df2['incorrect_feed']!=-1]['incorrect_feed'].values.astype('float32').mean())
    vect.append(df2[df2['time_corr_feed']!=-1]['time_corr_feed'].values.astype('float32').mean())
    vect.append(df2[df2['time_inc_feed']!=-1]['time_inc_feed'].values.astype('float32').mean())
    vect.append(df2[df2['acc_feed']!=-1]['acc_feed'].values.astype('float32').mean())
    vect.append(df2[df2['accuracy']!=-1]['accuracy'].values.astype('float32').mean())
    vect.append(df2[df2['n_positive']!=-1]['n_positive'].values.astype('float32').mean())
    vect.append(df2[df2['n_negative']!=-1]['n_negative'].values.astype('float32').mean())
    vect.append(df2[df2['movies']!=-1]['movies'].values.astype('float32').sum())
    vect.append(df2[df2['skipped_movie']!=-1]['skipped_movie'].values.astype('float32').sum())
    vect.append(df2[df2['tuto']!=-1]['tuto'].values.astype('float32').sum())
    vect.append(df2[df2['skipped_tuto']!=-1]['skipped_tuto'].values.astype('float32').mean())
    
    ## Activity done
    for x in titles:
        vect.append( df1[df1['title']==x].shape[0])
    
    
    return vect

columns = [
    'game_session',
'installation_id',
'title',
'type',
'world',
'timestamp',
'accuracy_group',
'accuracy',
'n_positive',
'n_negative',
'time_instruction',
'actions',
'total_time',
'start_rounds',
'end_rounds',
'action_rounds',
'time_rounds',
'hints',
'correct_feed',
'incorrect_feed',
'acc_feed',
'time_corr_feed',
'time_inc_feed',
'movies',
'skipped_movie',
'tuto',
'skipped_tuto'
]

cols = [
    'pred_title',
'n_clip',
'n_activity',
'Activity_time_instruction',
'Activity_total_time',
'Activity_actions',
'Activity_start_rounds',
'Activity_end_rounds',
'Activity_action_rounds',
'Activity_time_rounds',
'Activity_hints',
'same_title',
'n_assessment',
'Assessment_accuracy_group',
'Assessment_time_instruction',
'Assessment_total_time',
'Assessment_actions',
'Assessment_start_rounds',
'Assessment_end_rounds',
'Assessment_action_rounds',
'Assessment_time_rounds',
'Assessment_hints',
'Assessment_correct_feed',
'Assessment_incorrect_feed',
'Assessment_time_corr_feed',
'Assessment_time_inc_feed',
'Assessment_acc_feed',
'Assessment_accuracy',
'Assessment_n_positive',
'Assessment_n_negative',
'n_games',
'Games_accuracy_group',
'Games_time_instruction',
'Games_total_time',
'Games_actions',
'Games_start_rounds',
'Games_end_rounds',
'Games_action_rounds',
'Games_time_rounds',
'Games_hints',
'Games_correct_feed',
'Games_incorrect_feed',
'Games_time_corr_feed',
'Games_time_inc_feed',
'Games_acc_feed',
'Games_accuracy',
'Games_n_positive',
'Games_n_negative',
'Games_movies',
'Games_skipped_movie',
'Games_tuto',
'Games_skipped_tuto',  
]

for elt in titles:
    cols.append('actitivity_title_'+str(elt))

In [ ]:
data = load('data by session')
labels = pd.read_csv(train_labels)

In [ ]:
data.head()

In [ ]:
data[data['type'] == 'Assessment']['title'].unique()

In [ ]:
# dtitle = categorise(data['title'])
# dtype = categorise(data['type'])
# dworld = categorise(data['world'])

# # data = data.replace({'title' : dtitle})
# data = data.replace({'type' : dtype})
# data = data.replace({'world' : dworld})

data['date'] = data['timestamp'].apply(dateutil.parser.parse)


data = pd.concat([data,pd.get_dummies(data['type'],prefix=['type'])], axis = 1)
data = pd.concat([data,pd.get_dummies(data['title'],prefix=['title'])], axis = 1)
data = pd.concat([data,pd.get_dummies(data['world'],prefix=['world'])], axis = 1)
data = pd.concat([data,pd.get_dummies(data['accuracy_group'],prefix=['accuracy_group'])], axis = 1)


save((dtitle, dtype, dworld), 'dicos')

In [ ]:
data

In [ ]:
list(data.columns)

In [ ]:
columns1 = [
 'accuracy_group',
 'accuracy',
 'n_positive',
 'n_negative',
 'time_instruction',
 'actions',
 'total_time',
 'start_rounds',
 'end_rounds',
 'action_rounds',
 'time_rounds',
 'hints',
 'correct_feed',
 'incorrect_feed',
 'acc_feed',
 'time_corr_feed',
 'time_inc_feed',
 'movies',
 'skipped_movie',
 'tuto',
 'skipped_tuto',
 "['type']_Activity",
 "['type']_Assessment",
 "['type']_Clip",
 "['type']_Game",
 "['title']_0",
 "['title']_1",
 "['title']_2",
 "['title']_3",
 "['title']_4",
 "['title']_5",
 "['title']_6",
 "['title']_7",
 "['title']_8",
 "['title']_9",
 "['title']_10",
 "['title']_11",
 "['title']_12",
 "['title']_13",
 "['title']_14",
 "['title']_15",
 "['title']_16",
 "['title']_17",
 "['title']_18",
 "['title']_19",
 "['title']_20",
 "['title']_21",
 "['title']_22",
 "['title']_23",
 "['title']_24",
 "['title']_25",
 "['title']_26",
 "['title']_27",
 "['title']_28",
 "['title']_29",
 "['title']_30",
 "['title']_31",
 "['title']_32",
 "['title']_33",
 "['title']_34",
 "['title']_35",
 "['title']_36",
 "['title']_37",
 "['title']_38",
 "['title']_39",
 "['title']_40",
 "['title']_41",
 "['title']_42",
 "['title']_43",
 "['world']_CRYSTALCAVES",
 "['world']_MAGMAPEAK",
 "['world']_NONE",
 "['world']_TREETOPCITY",
 "['accuracy_group']_-1",
 "['accuracy_group']_0",
 "['accuracy_group']_1",
 "['accuracy_group']_2",
 "['accuracy_group']_3"]
dataset = list(np.zeros(labels.shape[0]))

for i in range(labels.shape[0]):
    if i%100 == 0:
        print(i)
    installation_id = labels.iloc[i]['installation_id']
    game_session = labels.iloc[i]['game_session']
    time = data[(data['installation_id'] == installation_id)&(data['game_session']==game_session)].iloc[0]['date']
    ass_title = data[(data['installation_id'] == installation_id)&(data['game_session']==game_session)].iloc[0]['title']
    
    df1 = data[(data['installation_id']==installation_id)&(data['date']<time)]
    df1 = df1.sort_values(by = ['date'])
    
    
    if ass_title == 28:
        df1['title_28'] = 1
        df1['title_24'] = 0
        df1['title_36'] = 0
        df1['title_14'] = 0
        df1['title_27'] = 0
    elif ass_title == 24:
        df1['title_28'] = 0
        df1['title_24'] = 1
        df1['title_36'] = 0
        df1['title_14'] = 0
        df1['title_27'] = 0
    elif ass_title == 36:
        df1['title_28'] = 0
        df1['title_24'] = 0
        df1['title_36'] = 1
        df1['title_14'] = 0
        df1['title_27'] = 0
    elif ass_title == 14:
        df1['title_28'] = 0
        df1['title_24'] = 0
        df1['title_36'] = 0
        df1['title_14'] = 1
        df1['title_27'] = 0
    else:
        df1['title_28'] = 0
        df1['title_24'] = 0
        df1['title_36'] = 0
        df1['title_14'] = 0
        df1['title_27'] = 1
    df1['pred_type'] = ass_type
    
    dataset[i] = df1[columns1]

save(dataset, 'data for lstm')

In [ ]:
M = 0
s = 0
for i in dataset:
    s+=i.shape[0]
    if i.shape[0] > M:
        M = i.shape[0]
s = s/17600

In [ ]:
M

In [ ]:
s

In [ ]:
dataset[0].values.astype('float32')

In [29]:
dataset = load('data for lstm')

pad = 150

dataset = padding(dataset,pad)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600


In [30]:
labels = pd.read_csv(train_labels)
X = np.array(dataset)
Y = labels['accuracy_group'].values
y = np_utils.to_categorical(Y)

from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(Y),
                                               Y)

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)

In [32]:
from keras.models import Model, Sequential
from keras.layers import Input, Dropout, Dense,CuDNNLSTM, Flatten, Embedding, TimeDistributed, Concatenate, LSTM, BatchNormalization, Lambda, Reshape
from keras.regularizers import l2

model = Sequential()

model.add(CuDNNLSTM(64, return_sequences = False, input_shape = (pad, 78)))
model.add(Dropout(0.2))
model.add(Dense(32, activation = 'relu',  kernel_initializer='normal'))
model.add(Dropout(0.2))
model.add(Dense(4,  kernel_initializer='normal', activation = 'sigmoid' ))

In [35]:
import tensorflow as tf

def kappa_loss(y_pred, y_true, y_pow=2, eps=1e-10, N=4, bsize=256, name='kappa'):
    """A continuous differentiable approximation of discrete kappa loss.
        Args:
            y_pred: 2D tensor or array, [batch_size, num_classes]
            y_true: 2D tensor or array,[batch_size, num_classes]
            y_pow: int,  e.g. y_pow=2
            N: typically num_classes of the model
            bsize: batch_size of the training or validation ops
            eps: a float, prevents divide by zero
            name: Optional scope/name for op_scope.
        Returns:
            A tensor with the kappa loss."""

    with tf.name_scope(name):
        y_true = tf.to_float(y_true)
        repeat_op = tf.to_float(tf.tile(tf.reshape(tf.range(0, N), [N, 1]), [1, N]))
        repeat_op_sq = tf.square((repeat_op - tf.transpose(repeat_op)))
        weights = repeat_op_sq / tf.to_float((N - 1) ** 2)
    
        pred_ = y_pred ** y_pow
        try:
            pred_norm = pred_ / (eps + tf.reshape(tf.reduce_sum(pred_, 1), [-1, 1]))
        except Exception:
            pred_norm = pred_ / (eps + tf.reshape(tf.reduce_sum(pred_, 1), [bsize, 1]))
    
        hist_rater_a = tf.reduce_sum(pred_norm, 0)
        hist_rater_b = tf.reduce_sum(y_true, 0)
    
        conf_mat = tf.matmul(tf.transpose(pred_norm), y_true)
    
        nom = tf.reduce_sum(weights * conf_mat)
        denom = tf.reduce_sum(weights * tf.matmul(
            tf.reshape(hist_rater_a, [N, 1]), tf.reshape(hist_rater_b, [1, N])) /
                              tf.to_float(bsize))
    
        return nom / (denom + eps)

In [36]:
model.compile(loss=kappa_loss, optimizer='adam')
batch_size = 12
epochs = 15

history = model.fit(X_train, y_train, batch_size = batch_size, epochs = epochs, validation_data=(X_test, y_test), class_weight=class_weights)

Train on 14152 samples, validate on 3538 samples
Epoch 1/15
14152/14152 [==============================] - ETA: 14:33 - loss: 21.350 - ETA: 2:37 - loss: 21.339 - ETA: 1:40 - loss: 21.33 - ETA: 1:11 - loss: 21.34 - ETA: 59s - loss: 21.3401 - ETA: 49s - loss: 21.338 - ETA: 42s - loss: 21.333 - ETA: 38s - loss: 21.331 - ETA: 35s - loss: 21.329 - ETA: 32s - loss: 21.326 - ETA: 30s - loss: 21.324 - ETA: 29s - loss: 21.327 - ETA: 27s - loss: 21.331 - ETA: 26s - loss: 21.337 - ETA: 25s - loss: 21.338 - ETA: 24s - loss: 21.338 - ETA: 23s - loss: 21.337 - ETA: 22s - loss: 21.337 - ETA: 22s - loss: 21.336 - ETA: 21s - loss: 21.333 - ETA: 21s - loss: 21.332 - ETA: 20s - loss: 21.333 - ETA: 20s - loss: 21.332 - ETA: 19s - loss: 21.332 - ETA: 19s - loss: 21.333 - ETA: 19s - loss: 21.334 - ETA: 18s - loss: 21.334 - ETA: 18s - loss: 21.334 - ETA: 18s - loss: 21.333 - ETA: 17s - loss: 21.332 - ETA: 17s - loss: 21.330 - ETA: 17s - loss: 21.330 - ETA: 17s - loss: 21.332 - ETA: 16s - loss: 21.332 - ETA: 

14152/14152 [==============================] - ETA: 15s - loss: 15.966 - ETA: 13s - loss: 9.620 - ETA: 13s - loss: 14.728 - ETA: 13s - loss: 13.767 - ETA: 12s - loss: 14.480 - ETA: 12s - loss: 13.936 - ETA: 12s - loss: 12.856 - ETA: 12s - loss: 12.734 - ETA: 12s - loss: 12.394 - ETA: 12s - loss: 12.393 - ETA: 12s - loss: 12.848 - ETA: 12s - loss: 13.042 - ETA: 12s - loss: 12.806 - ETA: 12s - loss: 12.629 - ETA: 12s - loss: 12.584 - ETA: 12s - loss: 12.335 - ETA: 12s - loss: 12.182 - ETA: 12s - loss: 11.954 - ETA: 12s - loss: 12.056 - ETA: 12s - loss: 11.835 - ETA: 12s - loss: 11.741 - ETA: 12s - loss: 11.971 - ETA: 11s - loss: 12.281 - ETA: 11s - loss: 12.276 - ETA: 11s - loss: 12.327 - ETA: 11s - loss: 12.321 - ETA: 11s - loss: 12.432 - ETA: 11s - loss: 12.387 - ETA: 11s - loss: 12.702 - ETA: 11s - loss: 12.925 - ETA: 11s - loss: 12.928 - ETA: 11s - loss: 12.626 - ETA: 11s - loss: 12.960 - ETA: 11s - loss: 12.761 - ETA: 11s - loss: 12.656 - ETA: 11s - loss: 12.730 - ETA: 11s - loss: 1

14152/14152 [==============================] - ETA: 15s - loss: 3.66 - ETA: 13s - loss: 11.984 - ETA: 13s - loss: 14.649 - ETA: 13s - loss: 15.570 - ETA: 13s - loss: 12.850 - ETA: 13s - loss: 13.239 - ETA: 13s - loss: 11.780 - ETA: 13s - loss: 11.870 - ETA: 13s - loss: 11.217 - ETA: 12s - loss: 11.428 - ETA: 12s - loss: 10.822 - ETA: 12s - loss: 11.296 - ETA: 12s - loss: 11.269 - ETA: 12s - loss: 11.480 - ETA: 12s - loss: 11.222 - ETA: 12s - loss: 10.756 - ETA: 12s - loss: 12.000 - ETA: 12s - loss: 12.495 - ETA: 12s - loss: 12.485 - ETA: 12s - loss: 12.692 - ETA: 11s - loss: 12.553 - ETA: 11s - loss: 13.565 - ETA: 11s - loss: 13.641 - ETA: 11s - loss: 13.523 - ETA: 11s - loss: 13.448 - ETA: 11s - loss: 14.318 - ETA: 11s - loss: 14.322 - ETA: 11s - loss: 14.284 - ETA: 11s - loss: 14.421 - ETA: 11s - loss: 14.132 - ETA: 11s - loss: 13.879 - ETA: 11s - loss: 14.367 - ETA: 11s - loss: 14.296 - ETA: 11s - loss: 14.288 - ETA: 11s - loss: 14.173 - ETA: 11s - loss: 14.052 - ETA: 11s - loss: 14

14152/14152 [==============================] - ETA: 17s - loss: 24.559 - ETA: 14s - loss: 13.833 - ETA: 14s - loss: 12.551 - ETA: 14s - loss: 13.630 - ETA: 13s - loss: 13.093 - ETA: 13s - loss: 11.676 - ETA: 13s - loss: 11.574 - ETA: 13s - loss: 11.453 - ETA: 13s - loss: 10.344 - ETA: 13s - loss: 10.597 - ETA: 13s - loss: 10.442 - ETA: 13s - loss: 10.675 - ETA: 13s - loss: 11.562 - ETA: 12s - loss: 11.390 - ETA: 12s - loss: 12.056 - ETA: 12s - loss: 12.563 - ETA: 12s - loss: 12.529 - ETA: 12s - loss: 12.149 - ETA: 12s - loss: 12.108 - ETA: 12s - loss: 11.977 - ETA: 12s - loss: 12.182 - ETA: 12s - loss: 11.677 - ETA: 12s - loss: 11.811 - ETA: 12s - loss: 11.470 - ETA: 12s - loss: 11.263 - ETA: 12s - loss: 11.223 - ETA: 12s - loss: 11.382 - ETA: 12s - loss: 11.111 - ETA: 12s - loss: 10.895 - ETA: 12s - loss: 10.584 - ETA: 11s - loss: 10.919 - ETA: 11s - loss: 10.944 - ETA: 11s - loss: 10.985 - ETA: 11s - loss: 10.856 - ETA: 11s - loss: 10.867 - ETA: 11s - loss: 10.877 - ETA: 11s - loss: 

14152/14152 [==============================] - ETA: 16s - loss: 9.96 - ETA: 13s - loss: 11.868 - ETA: 13s - loss: 11.776 - ETA: 13s - loss: 9.441 - ETA: 13s - loss: 8.91 - ETA: 13s - loss: 8.05 - ETA: 13s - loss: 9.23 - ETA: 12s - loss: 9.65 - ETA: 12s - loss: 9.61 - ETA: 12s - loss: 10.110 - ETA: 12s - loss: 10.434 - ETA: 12s - loss: 11.104 - ETA: 12s - loss: 11.783 - ETA: 12s - loss: 13.389 - ETA: 12s - loss: 12.531 - ETA: 12s - loss: 11.779 - ETA: 12s - loss: 11.857 - ETA: 12s - loss: 11.658 - ETA: 12s - loss: 11.115 - ETA: 12s - loss: 10.983 - ETA: 12s - loss: 10.815 - ETA: 12s - loss: 10.793 - ETA: 12s - loss: 10.547 - ETA: 11s - loss: 10.373 - ETA: 11s - loss: 10.581 - ETA: 11s - loss: 11.117 - ETA: 11s - loss: 10.939 - ETA: 11s - loss: 10.808 - ETA: 11s - loss: 10.959 - ETA: 11s - loss: 10.770 - ETA: 11s - loss: 10.707 - ETA: 11s - loss: 11.253 - ETA: 11s - loss: 11.674 - ETA: 11s - loss: 12.315 - ETA: 11s - loss: 12.186 - ETA: 11s - loss: 12.077 - ETA: 11s - loss: 12.056 - ETA:

...

14152/14152 [==============================] - ETA: 15s - loss: 1.02 - ETA: 13s - loss: 1.28 - ETA: 13s - loss: 3.47 - ETA: 13s - loss: 6.80 - ETA: 13s - loss: 6.39 - ETA: 13s - loss: 8.00 - ETA: 13s - loss: 7.26 - ETA: 13s - loss: 6.90 - ETA: 13s - loss: 7.96 - ETA: 13s - loss: 7.81 - ETA: 13s - loss: 8.03 - ETA: 12s - loss: 8.54 - ETA: 12s - loss: 7.96 - ETA: 12s - loss: 7.59 - ETA: 12s - loss: 7.87 - ETA: 12s - loss: 8.34 - ETA: 12s - loss: 9.06 - ETA: 12s - loss: 9.00 - ETA: 12s - loss: 8.84 - ETA: 12s - loss: 8.71 - ETA: 12s - loss: 9.06 - ETA: 12s - loss: 9.01 - ETA: 12s - loss: 8.89 - ETA: 12s - loss: 9.08 - ETA: 12s - loss: 8.94 - ETA: 12s - loss: 9.36 - ETA: 12s - loss: 9.46 - ETA: 12s - loss: 9.31 - ETA: 12s - loss: 9.68 - ETA: 12s - loss: 9.56 - ETA: 12s - loss: 9.63 - ETA: 12s - loss: 9.54 - ETA: 12s - loss: 9.31 - ETA: 12s - loss: 9.53 - ETA: 12s - loss: 9.36 - ETA: 11s - loss: 9.59 - ETA: 11s - loss: 9.39 - ETA: 11s - loss: 9.30 - ETA: 11s - loss: 9.44 - ETA: 11s - loss: 

14152/14152 [==============================] - ETA: 16s - loss: 2.53 - ETA: 13s - loss: 9.95 - ETA: 13s - loss: 10.030 - ETA: 13s - loss: 8.735 - ETA: 13s - loss: 8.44 - ETA: 13s - loss: 7.74 - ETA: 13s - loss: 9.16 - ETA: 13s - loss: 10.565 - ETA: 13s - loss: 10.005 - ETA: 13s - loss: 9.654 - ETA: 12s - loss: 9.84 - ETA: 12s - loss: 9.98 - ETA: 12s - loss: 10.382 - ETA: 12s - loss: 10.088 - ETA: 12s - loss: 9.559 - ETA: 12s - loss: 9.48 - ETA: 12s - loss: 9.31 - ETA: 12s - loss: 9.32 - ETA: 12s - loss: 9.59 - ETA: 11s - loss: 9.41 - ETA: 11s - loss: 9.02 - ETA: 11s - loss: 8.92 - ETA: 11s - loss: 8.67 - ETA: 11s - loss: 8.85 - ETA: 11s - loss: 8.83 - ETA: 11s - loss: 8.61 - ETA: 11s - loss: 8.45 - ETA: 11s - loss: 8.22 - ETA: 11s - loss: 8.07 - ETA: 11s - loss: 8.21 - ETA: 11s - loss: 8.25 - ETA: 10s - loss: 8.18 - ETA: 10s - loss: 8.33 - ETA: 10s - loss: 8.29 - ETA: 10s - loss: 8.44 - ETA: 10s - loss: 8.35 - ETA: 10s - loss: 8.33 - ETA: 10s - loss: 8.29 - ETA: 10s - loss: 8.24 - ETA:

14152/14152 [==============================] - ETA: 14s - loss: 10.835 - ETA: 12s - loss: 8.711 - ETA: 12s - loss: 10.564 - ETA: 12s - loss: 7.968 - ETA: 12s - loss: 7.25 - ETA: 12s - loss: 6.11 - ETA: 12s - loss: 6.18 - ETA: 12s - loss: 6.35 - ETA: 12s - loss: 5.80 - ETA: 11s - loss: 5.34 - ETA: 11s - loss: 6.43 - ETA: 11s - loss: 7.27 - ETA: 11s - loss: 7.27 - ETA: 11s - loss: 7.55 - ETA: 11s - loss: 7.23 - ETA: 11s - loss: 7.01 - ETA: 11s - loss: 7.34 - ETA: 11s - loss: 7.46 - ETA: 11s - loss: 7.83 - ETA: 11s - loss: 8.11 - ETA: 11s - loss: 8.12 - ETA: 11s - loss: 8.36 - ETA: 11s - loss: 8.29 - ETA: 11s - loss: 8.95 - ETA: 11s - loss: 9.56 - ETA: 11s - loss: 9.60 - ETA: 11s - loss: 9.66 - ETA: 11s - loss: 9.87 - ETA: 11s - loss: 9.88 - ETA: 10s - loss: 9.87 - ETA: 10s - loss: 9.77 - ETA: 10s - loss: 9.66 - ETA: 10s - loss: 9.58 - ETA: 10s - loss: 9.68 - ETA: 10s - loss: 9.52 - ETA: 10s - loss: 9.70 - ETA: 10s - loss: 9.79 - ETA: 10s - loss: 9.67 - ETA: 10s - loss: 9.77 - ETA: 10s - 

 8184/14152 [================>.............] - ETA: 12s - loss: 0.07 - ETA: 12s - loss: 6.09 - ETA: 12s - loss: 11.007 - ETA: 12s - loss: 10.854 - ETA: 12s - loss: 11.346 - ETA: 12s - loss: 14.100 - ETA: 12s - loss: 13.275 - ETA: 12s - loss: 11.658 - ETA: 12s - loss: 10.959 - ETA: 12s - loss: 11.326 - ETA: 12s - loss: 10.807 - ETA: 12s - loss: 10.018 - ETA: 12s - loss: 10.338 - ETA: 12s - loss: 10.435 - ETA: 12s - loss: 10.722 - ETA: 12s - loss: 11.031 - ETA: 12s - loss: 10.894 - ETA: 12s - loss: 10.587 - ETA: 12s - loss: 10.433 - ETA: 12s - loss: 10.207 - ETA: 12s - loss: 9.995 - ETA: 12s - loss: 10.085 - ETA: 11s - loss: 10.372 - ETA: 11s - loss: 10.609 - ETA: 11s - loss: 10.756 - ETA: 11s - loss: 10.609 - ETA: 11s - loss: 10.551 - ETA: 11s - loss: 10.589 - ETA: 11s - loss: 10.393 - ETA: 11s - loss: 11.019 - ETA: 11s - loss: 10.906 - ETA: 11s - loss: 11.133 - ETA: 11s - loss: 11.164 - ETA: 11s - loss: 11.282 - ETA: 11s - loss: 11.631 - ETA: 11s - loss: 11.512 - ETA: 11s - loss: 11.23

KeyboardInterrupt: 

In [37]:
pred = model.predict_classes(X_test)

In [38]:
pred

array([3, 0, 3, ..., 0, 3, 3], dtype=int64)

In [40]:
np.argmax(y_test, axis = 1)

array([3, 1, 0, ..., 0, 1, 3], dtype=int64)

In [41]:
cohen_kappa_score(np.argmax(y_test, axis = 1), pred, weights = 'quadratic')

0.3079428653383871

In [28]:
import optuna
def tres(pred, t):
    pred1 = deepcopy(pred)
    pred1[pred1 < t[0]] = 0
    pred1[(pred1>= t[0])&(pred1 < t[1])] = 1
    pred1[(pred1>= t[1])&(pred1 < t[2])] = 2
    pred1[pred1 >= t[2]] = 3
    
    return pred1

def objective(trial, pred = pred, y_test = y_test):
    x0 = trial.suggest_uniform('x0', pred.min(), pred.max())
    x1 = trial.suggest_uniform('x1', x0,  pred.max())
    x2 = trial.suggest_uniform('x2', x1,  pred.max())

    t = [x0, x1, x2]
    pred1 = tres(pred, t)
    a = cohen_kappa_score(y_test, pred1, weights = 'quadratic')
    print(a)
    return 1-a

study = optuna.create_study()
study.optimize(objective, n_trials=300)

0.25912366797539466


[I 2019-12-14 23:25:03,058] Finished trial#0 resulted in value: 0.7408763320246053. Current best value is 0.7408763320246053 with parameters: {'x0': 1.6242941500057828, 'x1': 2.147711414379804, 'x2': 2.8431700416223986}.


0.09689583081445652


[I 2019-12-14 23:25:03,219] Finished trial#1 resulted in value: 0.9031041691855435. Current best value is 0.7408763320246053 with parameters: {'x0': 1.6242941500057828, 'x1': 2.147711414379804, 'x2': 2.8431700416223986}.


0.31142511707749976


...

0.3986817517105711


[I 2019-12-14 23:25:30,346] Finished trial#147 resulted in value: 0.6013182482894289. Current best value is 0.5801400570498451 with parameters: {'x0': 1.225154658728667, 'x1': 1.558322153723163, 'x2': 2.075248203274682}.


0.4168811887893571


[I 2019-12-14 23:25:30,548]

KeyboardInterrupt: 